# ЗАДАНИЕ 9.5. МОДУЛЬ ML-6 (HW-03)

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. Загрузите полученный ноутбук (в формате IPYNB) в форму ниже.

In [81]:
# Импортируем нужные библиотеки и модели
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression


## 1. Загружаем и изучаем данные

In [82]:
# Загружаем данные
data = pd.read_excel('data_ford_price.xlsx', engine='openpyxl') 
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [84]:
# Выделим целевой признак
y = data['price']
x = data.drop(columns='price',inplace=False)

## 2. Заполним пропуски в данных

In [85]:
# Предскажем пропущенные данные непрерывного признака weather
 
test_data = x[x['weather'].isnull()]
x_test = x.dropna()
 
y_train = x_test['weather']
X_train = x_test.drop(['size','weather','drive'], axis=1)
X_test = test_data.drop(['size','weather','drive'], axis=1)
 
one_hot_encoder = OneHotEncoder()
categorial_cols = ['cylinders', 'title_status', 'transmission']
 
X_train_onehot = one_hot_encoder.fit_transform(X_train[categorial_cols]).toarray()
X_test_onehot = one_hot_encoder.transform(X_test[categorial_cols]).toarray()
 
columns = one_hot_encoder.get_feature_names(categorial_cols)
X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)
 
X_train = X_train.reset_index().drop(['index'], axis = 1)
X_test = X_test.reset_index().drop(['index'], axis = 1)
y_train = y_train.reset_index().drop(['index'], axis = 1)
 
X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)
 
X_train_new = X_train_new.drop(columns=categorial_cols)
X_test_new = X_test_new.drop(columns=categorial_cols)
 
 
model = LinearRegression()
model.fit(X_train_new, y_train)
 
y_pred = model.predict(X_test_new)


# Заменим пропущенные значения на предсказанные

for i, index in enumerate(test_data.index):
    x['weather'].iloc[index] = y_pred[i]

c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Home\AppData\Local\Temp\ipykernel_2400\2618340409.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['weather'].iloc[index] = y_pred[i]


In [86]:
# Функция однократного кодирования признаков

def encode_cat_features(columns_to_change, X_train, X_test, y_train):
  one_hot_encoder = OneHotEncoder()
  X_train_onehot = one_hot_encoder.fit_transform(X_train[columns_to_change]).toarray()
  X_test_onehot = one_hot_encoder.transform(X_test[columns_to_change]).toarray()

  columns = one_hot_encoder.get_feature_names(columns_to_change)
  
  X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
  X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)

  X_train = X_train.reset_index().drop(['index'], axis = 1)
  X_test = X_test.reset_index().drop(['index'], axis = 1)
  y_train = y_train.reset_index().drop(['index'], axis = 1)

  X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
  X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)
  
  X_train_new = X_train_new.drop(columns=columns_to_change)
  X_test_new = X_test_new.drop(columns=columns_to_change)

  return X_train_new, X_test_new

In [87]:
# Предскажем пропущенные данные катериального признака size

categorial_cols = ['cylinders', 'title_status', 'transmission']

test_data = x[x['size'].isnull()]
x_test = x.dropna()

y_train = x_test['size']
X_train = x_test.drop(['size', 'drive'], axis=1)
X_test = test_data.drop(['size', 'drive'], axis=1)

X_train_new, X_test_new = encode_cat_features(categorial_cols, X_train, X_test, y_train)

model = LogisticRegression()
model.fit(X_train_new, y_train)
 
y_pred = model.predict(X_test_new)
y_pred

# Заменим пропущенные значения на предсказанные
for i, index in enumerate(test_data.index):
    x['size'].iloc[index] = y_pred[i]

c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Home\AppData\Local\Temp\ipykernel_2400\1101700220.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [88]:
# Предскажем пропущенные данные катериального признака drive

categorial_cols = ['cylinders', 'title_status', 'transmission']

test_data = x[x['drive'].isnull()]
x_test = x.dropna()

y_train = x_test['drive']
X_train = x_test.drop(['size', 'drive'], axis=1)
X_test = test_data.drop(['size', 'drive'], axis=1)

X_train_new, X_test_new = encode_cat_features(categorial_cols, X_train, X_test, y_train)

model = LogisticRegression()
model.fit(X_train_new, y_train)
 
y_pred = model.predict(X_test_new)
y_pred

# Заменим пропущенные значения на предсказанные
for i, index in enumerate(test_data.index):
    x['drive'].iloc[index] = y_pred[i]

c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Home\AppData\Local\Temp\ipykernel_2400\313100432.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['drive'].iloc[index] = y_pred[i]


In [89]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          7017 non-null   int64  
 1   condition     7017 non-null   int64  
 2   cylinders     7017 non-null   int64  
 3   odometer      7017 non-null   int64  
 4   title_status  7017 non-null   object 
 5   transmission  7017 non-null   object 
 6   drive         7017 non-null   object 
 7   size          7017 non-null   object 
 8   lat           7017 non-null   float64
 9   long          7017 non-null   float64
 10  weather       7017 non-null   float64
dtypes: float64(3), int64(4), object(4)
memory usage: 603.1+ KB


In [90]:
x.head()

,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.000000
1,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.000000
2,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.000000
3,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.000000
4,2010,3,8,166062,clean,automatic,4wd,full-size,49.210949,-123.114720,40.914356


In [91]:
data_new = x
data_new['price'] = y

In [92]:
data_new.head()

,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather,price
0,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.000000,43900
1,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.000000,15490
2,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.000000,2495
3,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.000000,1300
4,2010,3,8,166062,clean,automatic,4wd,full-size,49.210949,-123.114720,40.914356,13865


## 3. Закодируем признаки

In [93]:
data_new.describe(include='object')

,title_status,transmission,drive,size
count,7017,7017,7017,7017
unique,5,3,3,4
top,clean,automatic,4wd,full-size
freq,6657,6708,5162,6134


In [94]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']

one_hot_encoder = OneHotEncoder()
 
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data_new[columns_to_change]).toarray()
 
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names(columns_to_change)

data_encoded = one_hot_encoder.fit_transform(data_new[columns_to_change]).toarray()

data_encoded = pd.DataFrame(
    data_encoded,
    index=data_new.index,
    columns=column_names
)

data_joined = data_new.join(
    data_encoded,
    how='left'
)

data_droped = data_joined.drop(columns=columns_to_change)

data_droped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    7017 non-null   int64  
 1   condition               7017 non-null   int64  
 2   odometer                7017 non-null   int64  
 3   lat                     7017 non-null   float64
 4   long                    7017 non-null   float64
 5   weather                 7017 non-null   float64
 6   price                   7017 non-null   int64  
 7   cylinders_3             7017 non-null   float64
 8   cylinders_4             7017 non-null   float64
 9   cylinders_5             7017 non-null   float64
 10  cylinders_6             7017 non-null   float64
 11  cylinders_8             7017 non-null   float64
 12  cylinders_10            7017 non-null   float64
 13  title_status_clean      7017 non-null   float64
 14  title_status_lien       7017 non-null   

c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## 4. Избавимся от выбросов

Вычислим МАЕ до удаления выбросов

In [95]:
data_dropna = data.dropna()

data_test = data_droped#[['price', 'year', 'condition', 'cylinders', 'odometer' ,'lat', 'long', 'weather']]
#data_test.dropna(inplace = True) 
 
y_num = data_test['price']
x_num = data_test.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x_num, y_num, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4831.815


Вычислим МАЕ после удаления выбросов

In [96]:
from sklearn.neighbors import LocalOutlierFactor
 
lof = LocalOutlierFactor(contamination=0.1)
y_predicted = lof.fit_predict(X_train)

mask = y_predicted != -1
X_train, y_train = X_train[mask], y_train[mask]
 
print(X_train.shape, y_train.shape)
 
model = LinearRegression()
model.fit(X_train, y_train)
 
y_predicted = model.predict(X_test)
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

(4420, 27) (4420,)
MAE: 4579.977


Значение МАЕ уменьшилось, поэтому применим метод LocalOutlierFactor для всего датасета. 

In [97]:
lof = LocalOutlierFactor(contamination=0.1)
data_predicted = lof.fit_predict(data_droped)

mask = data_predicted != -1
data_cleared = data_droped[mask]
 
print(data_cleared.shape, data_cleared.shape)

(6315, 28) (6315, 28)


## 5. Отмасштабируем признаки

In [98]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
 
data_scaled = scaler.fit_transform(data_cleared.values)

data_scaled = pd.DataFrame(
    data_scaled,
    index=data_cleared.index,
    columns=data_cleared.columns
)

In [99]:
data_scaled.head()

,year,condition,odometer,lat,long,weather,price,cylinders_3,cylinders_4,cylinders_5,...,transmission_automatic,transmission_manual,transmission_other,drive_4wd,drive_fwd,drive_rwd,size_compact,size_full-size,size_mid-size,size_sub-compact
1,0.830189,0.4,0.312519,0.536372,0.527013,0.476237,0.224482,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.698113,0.4,0.642685,0.561437,0.467513,0.410887,0.036145,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.660377,0.2,0.542373,0.540061,0.471632,0.448230,0.018826,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.849057,0.6,0.528860,0.645476,0.191801,0.372745,0.200930,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.716981,0.6,0.533955,0.599388,0.195480,0.457565,0.101364,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [100]:
data_price = data_scaled
data_price['price'] = data_cleared['price']

y = data_price['price']
x = data_price.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
 
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 3742.493


## 6. Обучаем регрессию на первых трёх столбцах, оцениваем качество модели на тесте.

In [101]:
# Обучаем регрессию на первых трёх столбцах
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
rfe_columns = selector.get_feature_names_out()
rfe_columns

array(['size_compact', 'size_full-size', 'size_sub-compact'], dtype=object)

In [102]:
# Оцениваем качество модели на тесте
y = data_price['price']
x = data_price[rfe_columns]


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
 
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 8535.327


## 6. Обучаем регрессию на вторых трёх столбцах, оцениваем качество модели на тесте.

In [103]:
y = data_price['price']
x = data_price.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [104]:
# Обучаем регрессию на первых трёх столбцах
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
skb_columns = selector.get_feature_names_out()
skb_columns

array(['year', 'condition', 'odometer'], dtype=object)

In [105]:
# Оцениваем качество модели на тесте
y = data_price['price']
x = data_price[skb_columns]


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
 
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4072.902


## Вывод

По результатам вычисления МАЕ целевого признака на обработанных данных метод определения наиболее полезных признаков SelectKBest показал гораздо более высокие результаты, чем метод RFE.

# Результаты решения задачи на сырых данных

In [106]:
# Удаляем наблюдения с пропусками
data_test = data.dropna()

In [107]:
# Удаляем столбцы со строчными данными
data_test = data_test.drop(columns=['title_status','transmission','drive','size'])
data_test.head()


,price,year,condition,cylinders,odometer,lat,long,weather
0,43900,2016,4,6,43500,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,40.764373,-82.349503,49.0
5,6995,2003,3,8,167662,45.518031,-122.578752,50.0


### Метод RFE

In [108]:
# Выделяем целевой признак
x = data_test.drop(columns='price')
y = data_test['price']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [109]:
# Обучаем регрессию на первых трёх столбцах
selector = RFE(estimator, n_features_to_select=3, step=1)
selector.fit(X_train, y_train)
 
rfe_columns = selector.get_feature_names_out()
rfe_columns

array(['year', 'condition', 'cylinders'], dtype=object)

In [110]:
# Оцениваем качество модели на тесте
y = data_test['price']
x = data_test[rfe_columns]


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4979.608


### Метод SelectKBest

In [111]:
# Выделяем целевой признак
x = data_test.drop(columns='price')
y = data_test['price']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [112]:
# Обучаем регрессию на первых трёх столбцах
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
skb_columns = selector.get_feature_names_out()
skb_columns

array(['year', 'condition', 'odometer'], dtype=object)

In [113]:
# Оцениваем качество модели на тесте
y = data_test['price']
x = data_test[skb_columns]


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4660.119


### Вывод:
На сырых данных метод определения наиболее полезных признаков SelectKBest показал более высокие результаты, чем метод RFE. На обработанных данных метод SelectKBest показал более высокий результат, а RFE гораздо более низкий.